In [ ]:
# This notebook is  used to evaluate perofrmance of our RAG ingestion and query pipeline.
# Method:
#   - Generated test data using a sample GitHub repository (offline process not in this notebook)
#     - https://docs.ragas.io/en/latest/getstarted/rag_testset_generation
#   - Use our ingestion pipeline to parse the same reporisiroty and index in our vector store
#   - Use RAGAS on teadt dataset + our answers fromn our RAG and visualise the mterics.
import os
import sys
import pandas as pd

sys.path.insert(1, '/home/jovyan/work/code')
from opentelemetry import trace
from config import VectorDBConfig, EmbeddingConfig, ProcessingConfig, ChatConfig
from config_helper import ConfigHelper
from pipeline import DocumentPipeline
from localrag import LocalRAG
import nltk

# We are using Aspire. Of course we will see the telemetry and logs in our dashboard!
# see config_helper.py for the not to tidy details.
tracer = trace.get_tracer(__name__)

config_helper = ConfigHelper(True)
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
!git clone https://github.com/dotnet/docs-aspire
directory = "docs-aspire"

In [15]:
from langchain_community.document_loaders import DirectoryLoader
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

path = "docs-aspire/"
loader = DirectoryLoader(path, glob="**/*.md")
docs = loader.load()

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

dataset.to_pandas()

Applying HeadlinesExtractor:   0%|          | 0/114 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/179 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/153 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/188 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/525 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Could you elaborate on how code changes can im...,[title: Compatibility description: Learn about...,Code changes can affect compatibility in .NET ...,single_hop_specifc_query_synthesizer
1,Is forward compatibility a goal of .NET Aspire?,[Design-time compatibility Backwards compatibi...,Maintaining forward compatibility is not a goa...,single_hop_specifc_query_synthesizer
2,How can a Cloud Application Developer utilize ...,[title: .NET Aspire PostgreSQL integration des...,To utilize the NpgsqlDataSource in .NET Aspire...,single_hop_specifc_query_synthesizer
3,what .NET Aspire GitHub repo do?,[Logging The .NET Aspire PostgreSQL integratio...,The .NET Aspire GitHub repo is mentioned in th...,single_hop_specifc_query_synthesizer
4,Howw doo I use mcr.microsoft.com/mssql/server ...,[title: .NET Aspire SQL Server integration des...,The .NET Aspire SQL Server integration allows ...,single_hop_specifc_query_synthesizer
5,How can .NET Aspire's testing and Orleans inte...,[<1-hop>\n\ntitle: Manage the app host in .NET...,"In .NET Aspire, the testing framework and Orle...",multi_hop_specific_query_synthesizer
6,How do you use AspNetCore.HealthChecks.Kafka a...,[<1-hop>\n\ntitle: .NET Aspire NATS integratio...,"In .NET Aspire applications, the health of Kaf...",multi_hop_specific_query_synthesizer
7,How does the DOTNET_RESOURCE_SERVICE_ENDPOINT_...,[<1-hop>\n\nInternal settings that .NET Aspire...,The DOTNET_RESOURCE_SERVICE_ENDPOINT_URL confi...,multi_hop_specific_query_synthesizer
8,How does the ASPNETCORE_URLS setting in launch...,[<1-hop>\n\nInternal settings that .NET Aspire...,"In .NET Aspire, the ASPNETCORE_URLS setting in...",multi_hop_specific_query_synthesizer
9,How do you use Azure Key Vault for securing se...,[<1-hop>\n\nLocal emulators Some Azure service...,"In .NET Aspire, Azure Key Vault is used to sec...",multi_hop_specific_query_synthesizer


In [18]:
import pandas as pd

type(dataset)
file_name= "test_data__aspire_15.pkl"
df = dataset.to_pandas()
df.to_pickle(file_name) 
df1 =  pd.read_pickle(file_name)
df1

,user_input,reference_contexts,reference,synthesizer_name
0,Could you elaborate on how code changes can im...,[title: Compatibility description: Learn about...,Code changes can affect compatibility in .NET ...,single_hop_specifc_query_synthesizer
1,Is forward compatibility a goal of .NET Aspire?,[Design-time compatibility Backwards compatibi...,Maintaining forward compatibility is not a goa...,single_hop_specifc_query_synthesizer
2,How can a Cloud Application Developer utilize ...,[title: .NET Aspire PostgreSQL integration des...,To utilize the NpgsqlDataSource in .NET Aspire...,single_hop_specifc_query_synthesizer
3,what .NET Aspire GitHub repo do?,[Logging The .NET Aspire PostgreSQL integratio...,The .NET Aspire GitHub repo is mentioned in th...,single_hop_specifc_query_synthesizer
4,Howw doo I use mcr.microsoft.com/mssql/server ...,[title: .NET Aspire SQL Server integration des...,The .NET Aspire SQL Server integration allows ...,single_hop_specifc_query_synthesizer
5,How can .NET Aspire's testing and Orleans inte...,[<1-hop>\n\ntitle: Manage the app host in .NET...,"In .NET Aspire, the testing framework and Orle...",multi_hop_specific_query_synthesizer
6,How do you use AspNetCore.HealthChecks.Kafka a...,[<1-hop>\n\ntitle: .NET Aspire NATS integratio...,"In .NET Aspire applications, the health of Kaf...",multi_hop_specific_query_synthesizer
7,How does the DOTNET_RESOURCE_SERVICE_ENDPOINT_...,[<1-hop>\n\nInternal settings that .NET Aspire...,The DOTNET_RESOURCE_SERVICE_ENDPOINT_URL confi...,multi_hop_specific_query_synthesizer
8,How does the ASPNETCORE_URLS setting in launch...,[<1-hop>\n\nInternal settings that .NET Aspire...,"In .NET Aspire, the ASPNETCORE_URLS setting in...",multi_hop_specific_query_synthesizer
9,How do you use Azure Key Vault for securing se...,[<1-hop>\n\nLocal emulators Some Azure service...,"In .NET Aspire, Azure Key Vault is used to sec...",multi_hop_specific_query_synthesizer


In [20]:
dataset = []

def demonstrate_local_rag(rag):
    """Demonstrate how to use the LocalRAG class."""    
    # Example questions to test
    questions = [
        "Why should I know about .Net Aspire?",
        "Is .Net Aspire an alternative to Kubernetes?"
    ]
    with tracer.start_as_current_span("Entering questions loop."):
        for question in questions:
            print(f"Question: {question}")
            print("\nGenerated Answer:")
            with tracer.start_as_current_span("Retrieve answers."):
                answer = rag.retrieve_and_answer(question, k=6)
                print(answer)
                print("\n" + "="*80 + "\n")

rag = LocalRAG(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config, 
    chat_config=config_helper.chat_config
)
with tracer.start_as_current_span("Starting demo"):
    demonstrate_local_rag(rag)


/home/jovyan/work/code/localrag.py:42: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)


UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `eval-rag-demo` doesn\'t exist!"},"time":6.375e-6}'